<a href="https://colab.research.google.com/github/qwesdme/AHCD-OCR-CNN-LSTM/blob/master/Appendix_OCR_AHCB_CNN_%2B_LSTM_with_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from pathlib import Path

import io
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from plotly import express as px
from plotly import graph_objects as go
from plotly import figure_factory as ff

from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

#Prepare datasets

In [0]:
# Load datasets
def git_file(file_name):
    return f"https://github.com/EyeBool/YAGAN/blob/master/data/Arabic/{file_name}?raw=true"

x_train_data = pd.read_csv(git_file("csvTrainImages%2013440x1024.csv"), header = None)
x_test_data = pd.read_csv(git_file("csvTestImages%203360x1024.csv"), header = None)

y_train_data = pd.read_csv(git_file("csvTrainLabel%2013440x1.csv"), header = None)
y_test_data = pd.read_csv(git_file("csvTestLabel%203360x1.csv"), header = None)

# Normalize datasets - set range from (0 - 255) to (0.0 - 1.0) 
x_train = (x_train_data.iloc[:,:].values.astype('float32') / 255).reshape([-1, 32, 32, 1])
x_test = (x_test_data.iloc[:,:].values.astype('float32') / 255).reshape([-1, 32, 32, 1])

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train_data - 1, 28)
y_test = tf.keras.utils.to_categorical(y_test_data - 1, 28)

# Print input shape
print("Input shape for images:", x_train.shape)
print("Input shape for labels:", y_train.shape)

In [0]:
f = px.imshow(np.transpose(x_train[0].reshape((32, 32))))
f.update_layout(width=480, height=480)
f.show()

def visualize_images(df, img_size, number_of_images):
    plt.figure(figsize=(8, 8))
    reshaped_df = df.reshape(df.shape[0], img_size, img_size)
    number_of_rows = number_of_images/4 if number_of_images%4 == 0 else (number_of_images/4) +1
    for i in range(number_of_images):
        plt.subplot(number_of_rows, 4, i+1, xticks=[], yticks=[])
        plt.imshow(np.transpose(reshaped_df[i * 4]), cmap='BuPu')

visualize_images(x_train, 32, 16)  # 32, image size, 16, num of image to show 

#Prepare training images 

In [0]:
# Split training images into fitting set and fit validation set  

x_fit = x_train[1000:]
y_fit = y_train[1000:]

x_fit_val = x_train[:1000]
y_fit_val = y_train[:1000]

print('Total images for fitting   :', len(x_fit))
print('Total images for validating: ', len(x_fit_val))

In [0]:
# Use Keras ImageDataGenerator to generate to image data with real-time data augmentation.

datagen = ImageDataGenerator(  
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False)  

datagen.fit(x_train)

#Create and train neural network models

In [0]:
# Set input shape
input_shape = (x_train.shape[1], x_train.shape[1], x_train.shape[3])
print(input_shape)

In [0]:
# Create function to compile and train model

def compile_and_train(nn_model):
    model: Sequential = nn_model

    model.compile(
        optimizer=Adam(amsgrad=True),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy']
    )


    learning_rate = ReduceLROnPlateau(
        monitor='val_categorical_accuracy', 
        patience=3, 
        verbose=1, 
        factor=9/10, 
        cooldown=2,
        min_delta=0.0005,
        min_lr=0.0005
        )
    
    history = model.fit_generator(
        datagen.flow(x_fit, y_fit, batch_size = 64),
        epochs = 30, 
        callbacks=[learning_rate],
        steps_per_epoch=x_train.shape[0] // 64,
        validation_data = (x_fit_val, y_fit_val),
        verbose=2
        )

In [0]:
def evaluate(nn_model):
    return nn_model.evaluate(x_test, y_test)

In [0]:
# Create function to evaluate model and plot chart

def evaluate_and_plot(nn_model, title):
    model = nn_model
    acc = evaluate(model)[1]*100

    cols =["loss", "categorical_accuracy", "val_loss", "val_categorical_accuracy", "lr"]
    col_names = ["Loss", "Categorical accuracy", "Validation loss", "Validation categorical accuracy", "learning rate (x 1000)"]
    names = {cols[i]: col_names[i] for i in range(len(cols))}

    x = pd.DataFrame(model.history.history, columns=cols)

    fig = go.Figure()

    for col in cols:
        fig.add_trace(go.Scatter(x=x.index, y= [i * 1000 for i in x[col]] if col is 'lr' else x[col], mode='lines', name=names[col]))
    fig.update_layout(title=f"{title} | Accuracy: {acc:.3f}%",
                    xaxis_title="Epoch",
                    yaxis_title="Rate",
    )
    return fig

In [0]:
# Build convolutional neural network (CNN) model

def build_CNN_model():
    return Sequential([
        # Input layer 
        Input(shape=input_shape),

        # Convolutional layer block 1
        Conv2D(64, 5, padding='same', activation="relu"),
        Conv2D(64, 5, activation="relu"),
        MaxPooling2D(2),
     
        # Convolutional layer block 2
        Conv2D(128, 3, padding='same', activation="relu"),
        Conv2D(128, 3, activation="relu"),
        MaxPooling2D(2),

        # Convolutional layer block 3
        Conv2D(256, 3, padding='same', activation="relu"),
        Conv2D(256, 3, activation="relu"),
        MaxPooling2D(2),

        #Dense layer
        Flatten(),
        Dense(1024, activation="relu"), 

        # Output layer
        Flatten(),
        Dense(28, activation="softmax"), 
    ])

CNN_model = build_CNN_model()

CNN_model.summary()

In [0]:
compile_and_train(CNN_model)

In [0]:
CNN_fig = evaluate_and_plot(CNN_model, "Training CNN")
CNN_fig.show()

In [0]:
# Build convolutional neural network (CNN) model with Dropout

def build_CNN_dropout_model():
    return Sequential([
        # Input layer 
        Input(shape=input_shape),

        # Convolutional layer block 1
        Conv2D(64, 5, padding='same', activation="relu"),
        Conv2D(64, 5, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),
     
        # Convolutional layer block 2
        Conv2D(128, 3, padding='same', activation="relu"),
        Conv2D(128, 3, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),

        # Convolutional layer block 3
        Conv2D(256, 3, padding='same', activation="relu"),
        Conv2D(256, 3, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),

        #Dense layer
        Flatten(),
        Dense(1024, activation="relu"), 
        Dropout(0.25),

        # Output layer
        Flatten(),
        Dense(28, activation="softmax"), 
    ])

CNN_dropout_model = build_CNN_dropout_model()

CNN_dropout_model.summary()


In [0]:
compile_and_train(CNN_dropout_model)

In [0]:
CNN_dropout_fig = evaluate_and_plot(CNN_dropout_model,"Training CNN (with Dropout)")
CNN_dropout_fig.show()

In [0]:
# Build long short term memory (LSTM) neural network model

def build_LSTM_model():
    return Sequential([
        # Input layer 
        Input(shape=input_shape),

        # LSTM layer
        Flatten(),
        Reshape((2, 512)),
        LSTM(280),
        Dropout(0.25),

        # Output layer
        Flatten(),
        Dense(28, activation="softmax"), 
    ])

LSTM_model = build_LSTM_model()

LSTM_model.summary()


In [0]:
compile_and_train(LSTM_model)

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (100 cells)")
LSTM_fig.show()

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (200 cells)")
LSTM_fig.show()

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (300 cells)")
LSTM_fig.show()

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (400 cells)")
LSTM_fig.show()

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (500 cells)")
LSTM_fig.show()

In [0]:
LSTM_fig = evaluate_and_plot(LSTM_model, "Training LSTM (1000 cells)")
LSTM_fig.show()

In [0]:
# Build hybrid neural network model

def build_hybrid_model():
    return Sequential([
        # Input layer 
        Input(shape=input_shape),

        # Convolutional layer block 1
        Conv2D(64, 5, padding='same', activation="relu"),
        Conv2D(64, 5, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),
     
        # Convolutional layer block 2
        Conv2D(128, 3, padding='same', activation="relu"),
        Conv2D(128, 3, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),

        # Convolutional layer block 3
        Conv2D(256, 3, padding='same', activation="relu"),
        Conv2D(256, 3, activation="relu"),
        MaxPooling2D(2),
        Dropout(0.25),

        #Dense layer
        Flatten(),
        Dense(1024, activation="relu"), 
        Dropout(0.25),

        # LSTM layer
        Flatten(),
        Reshape((2, 512)),
        LSTM(300, return_sequences=True),
        Dropout(0.25),

        # Output layer
        Flatten(),
        Dense(28, activation="softmax"), 
    ])


In [0]:
hybrid_model = build_hybrid_model()
compile_and_train(hybrid_model)
while(evaluate(hybrid_model)[1]*100) < 98.2:
    model = hybrid_model = build_hybrid_model()
    # hybrid_model.summary()
    compile_and_train(hybrid_model)

In [0]:
hybrid_fig = evaluate_and_plot(hybrid_model, "Training CNN + LSTM(300) (with Dropout)")
hybrid_fig.show()

#Result and evaluation

In [0]:
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np
plt.figure(num='digit',figsize=(9,9))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10))

In [0]:
Y_pred = model.predict(x_test)
Y_true = np.argmax(y_test,axis = 1) 

Y_pred_classes = np.argmax(Y_pred,axis = 1) 

tp = (Y_pred_classes - Y_true == 0)
fp = (Y_pred_classes - Y_true != 0)

fn = (Y_pred_classes - Y_true != 1)
tn = (Y_pred_classes - Y_true == 1)

print("True positive  :", list(tp).count(True))
print("False positive :", list(fp).count(True))
print("True negative  :", list(tn).count(True))
print("False negative :", list(fn).count(True))


In [0]:
#Display some error results 
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = x_test[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 4
    ncols = 4
    plt.figure(num='digit',figsize=(25,25))
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]

            im_err = np.transpose(img_errors[error].reshape((32, 32)))
            ax[row,col].imshow(im_err)
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-16:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

#Save trained neural network model

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from datetime import datetime
# Save neural network structure
model_structure = model.to_yaml()
f = Path(f"/content/drive/My Drive/Colab Notebooks/models/CNN_LSTM_model_structure{datetime.now()}{acc:.3f}.yaml")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights(f"/content/drive/My Drive/Colab Notebooks/models/CNN_LSTM_model_weights{datetime.now()}{acc:.3f}.h5")
print(f"Done saving {f}")